<h1 style="font-size: 36px; color: #FFD700">5 - BUILD the NEURAL NETWORK</h1>

Không gian tên torch.nn cung cấp tất cả các khối xây dựng mạng nơ-ron. Mỗi mô-đun trong PyTorch đều là lớp con của nn.Module . Mạng nơ-ron là một mô-đun bao gồm các mô-đun khác (lớp). Cấu trúc lồng nhau này cho phép xây dựng và quản lý các kiến ​​trúc phức tạp một cách dễ dàng.

In [3]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Device -  có thể đào tạo mô hình của mình trên một accelerator như CUDA, MPS, MTIA hoặc XPU. Nếu bộ tăng tốc hiện tại khả dụng, chúng tôi sẽ sử dụng nó. Nếu không, chúng tôi sẽ sử dụng CPU.

In [4]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


Define - class: phân lớp nn.Module và khởi tạo các lớp mạng nơ-ron trong __init__. Mỗi lớp con nn.Module thực hiện các thao tác trên dữ liệu đầu vào trong phương thức forward.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Tạo một models thể hiện của Neural network và chuyển nó đến device

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Để dùng mô hình, ta chuyển tiếp cho nó dữ liệu đầu vào. Điều này thực thi mô hình forward cùng với một số hđ nền. Không gọi forward trực tiếp.

Gọi mô hình trên đầu vào trả về một tensor 2 chiều với dim=0 tương ứng với mỗi đầu ra của 10 giá trị dự đoán thô cho mỗi lớp và dim=1 tương ứng với các giá trị riêng lẻ của mỗi đầu ra.
- ảnh ngẫu nhiên có size 28*28, batch__size = 1 (có 1 ảnh đầu vào)
- dim = 0 trục dọc -> mỗi hàng một mẫu ảnh 
- dim = 1 trục ngang -> mỗi cột là giá trị cho một lớp (0 - 9)

Chúng ta có được xác suất dự đoán bằng cách truyền nó qua một thể hiện của module nn.Softmax

In [10]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_prob = nn.Softmax(dim=1)(logits)
y_pred = pred_prob.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


Models layers

In [11]:
# lấy một minibatch gồm 3 hình ảnh có kích thước 28x28
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten để chuyển đổi hình 2D thành một mảng liền kề gồm 784 giá trị pixel - kích thước minibatch tại dum = 0 được duy trì 

In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear - áp dụng biến đổi tuyến tính vào đầu vào bằng cách sdung trọng số và độ lệch được lưu trữ của nó

In [15]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU kích hoạt phi tuyến tính tạo ra các ánh xạ giữa các đầu vào và đầu ra của mô hình. Chúng được áp dụng sau các phép biến đổi tuyến tính để đưa vào tính phi tuyến tính giúp mạng neural học được nhiều hiện tượng khác nhau. 

In [16]:
# Dùng hàm kích hoạt để đưa tính phi tuyến vào mô hình
print(f"Before ReLu: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLu: tensor([[ 0.4186,  0.3890, -0.2776, -0.2148,  0.0639,  0.0667,  0.4413,  0.1873,
         -0.0813, -0.0056,  0.4241,  0.3613,  0.0495, -0.1898, -0.3598, -0.4590,
          0.4191,  0.4478, -0.0819, -0.1512],
        [ 0.1971,  0.4311, -0.1787,  0.0085,  0.3031,  0.1306,  0.8370,  0.3507,
          0.1357,  0.3999,  0.6399, -0.0217,  0.3160, -0.3835,  0.1261, -0.3442,
          0.3675,  0.4035,  0.1043,  0.0423],
        [ 0.0811,  0.5748, -0.1728,  0.0425, -0.0480,  0.0639,  0.8966,  0.1570,
         -0.1531, -0.0447,  0.4791,  0.4385,  0.1223, -0.2272, -0.1948, -0.3357,
          0.3031, -0.0743,  0.2906, -0.0082]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.4186, 0.3890, 0.0000, 0.0000, 0.0639, 0.0667, 0.4413, 0.1873, 0.0000,
         0.0000, 0.4241, 0.3613, 0.0495, 0.0000, 0.0000, 0.0000, 0.4191, 0.4478,
         0.0000, 0.0000],
        [0.1971, 0.4311, 0.0000, 0.0085, 0.3031, 0.1306, 0.8370, 0.3507, 0.1357,
         0.3999, 0.6399, 0.0000, 0.3160, 0.0000, 0.12

nn.Sequential là một container có thứ tự của các module. Dữ liệu được truyền qua tất cả các module theo cùng thứ tự như đã xác định. Ta có thể dùng các container tuần tự để tạo thành một mạng nhanh như seq_module

In [18]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
) # Đang hiển thị 2 lớp học
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

nn.Softmax - lớp linear cuối cùng trả về logit được truyền đến module nn.Softmax. Logit được chia tỷ lệ thành các giá trị [0, 1] biễu diễn xs dự đoán của mô hình có từng lớp. dim là tham số chỉ ra chiều mà các giá trị tổng phải bằng 1

In [19]:
softmax = nn.Softmax(dim=1)
pred_prob = softmax(logits)

Một lớp mạng neural được tham số hóa - weights + bias có thể học được. Những tham số này được tối ưu hóa trong quá trình huấn luyện

In [20]:
print(f"Model structure: {model}\n\n")
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0096,  0.0243,  0.0113,  ...,  0.0034,  0.0095,  0.0169],
        [ 0.0265, -0.0235,  0.0126,  ..., -0.0067, -0.0331,  0.0336]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0010,  0.0065], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0299, -0.0352,  0.0060,  ...,  0.0153,  0.0371, -0.0375],
        [-0.0248,  0.0043,  0.0391,  ..., -0.0256,  0.0054,  0.0216]],
       device='cuda:0', grad_fn=<Slice